# CENG463 PA2

In this programming assignment, you will be dealing with word embeddings and neural networks. You will use Python for this task. You can use libraries such as `pandas`, `nltk`, `numpy` etc. for your implementations, or implement your own functions. However, you are expected to analyse and reason about your implementation and results. The assignment consists of 3 questions.

### IMPORTANT NOTE

Do not move or delete the given cells, only add cells inbetween the questions for your answers.

In [23]:
# UPDATE THIS CELL TO INSTALL NEEDED LIBRARIES.
# MAKE SURE TO ADD EVERYTHING THAT NEEDS TO BE INSTALLED IN THIS CELL!

# we will use pip to install packages - you can add others below
!pip install pandas
!pip install numpy
!pip install nltk
!pip install gensim
!pip install scikit-learn

# and import them here - you can add others below
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import numpy as np
from numpy.linalg import norm
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, roc_auc_score
)
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
)
import time
from sklearn.utils import class_weight
from tensorflow.keras.layers import Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Q1 - Word embeddings (50 points)

In this question, you will first train a Word2Vec model, then use it to represent and reason about user reviews.

### Q1.A - training (10 points)
Load the `user_review_train.csv` file shared with you. Using `Word2Vec` module of `gensim.models`, train a **skip-gram** Word2Vec model on the train data.

#### Notes and tips

- Use the given preprocessing function `preprocess_review`.

In [24]:
# PREPROCESSING FUNCTIONS GIVEN FOR YOU

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')  
nltk.download('stopwords')
nltk.download('punkt_tab')

def preprocess_review(review):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    sentences = sent_tokenize(review)
    
    lemmatized_review = []
    
    for sentence in sentences:
        tokenized_sentence = word_tokenize(sentence)
        lowercased_sentence = [token.lower() for token in tokenized_sentence]
        stopwords_removed_sentence = [token for token in lowercased_sentence if token not in stop_words]
        lemmatized_sentence = [lemmatizer.lemmatize(token) for token in stopwords_removed_sentence]
        
        lemmatized_review = lemmatized_review + lemmatized_sentence
    
    return lemmatized_review

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yarkinozcan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yarkinozcan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/yarkinozcan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [136]:
# Q1.A - implementation
# you can add cells below if needed

# Load the training data
train = pd.read_csv("data/user_review_train.csv") # review are on the 'review' column

# List for storing the tokenized reviews
tokenized_reviews = []

for review in train['review']:
    # preprocess each review using given function
    tokens = preprocess_review(review) # This list stores tokens for single review
    tokenized_reviews.append(tokens) # This list stores tokens for all reviews (a list of lists)

model = Word2Vec(
    sentences=tokenized_reviews, # A list of list where each list includes tokens for each review
    vector_size=100, # The number of dimensions in each embedding vector
    window=5, # The sliding window size -> this is for checking the context of a word
    min_count=3, # words occuring less than 1 times are ignored
    workers=4, # This is the number of cores that will be used in training -> to accelerate training 
    sg=1, # This is for enabling skip-gram
    epochs=5, # The number of times model goes through the dataset -> I tried increasing this but it made all similirity scores go really low possibly causing an overfit
)

print('training complete') # I added this to track if the training is done

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


training complete


### Q1.B - word similarity (10 points)

Using the trained model, report the following:

- Similarity between "good" and "bad"
- Similar words to "good"
- Similar words to "bad"
- Similar words to "good" but not similar to "bad"
- Similar words to "good" but not similar to "bad"

and discuss the reported words and scores. Is it possible to identify specific good/bad features of the product that is being reviewed? What other words can be looked up to get more insight?

#### Notes and tips

- Check the [documentation](https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.html) of `gensim.models.Word2Vec` to find relevant methods.

In [137]:
# Q1.B - implementation
# you can add cells below if needed

# Check the similarity score between 'good' and 'bad'
# Here in order to reach word vectors I had to use .wv
similarity_score = model.wv.similarity('good', 'bad') # Found a .similarity(w1, w2) method in the documentation
print(f'Similarity between "good" and "bad": {similarity_score:.4f}\n')

# Check the mostSimilar words to 'good'
print("Top similar words to 'good':") 
# Found a .most_similar method in the documentation
# I kept topn parameter as default (value is 10)
# The return value is a 2-tuple (word-score pairs)
most_similar_good = model.wv.most_similar(positive=["good"]) 
for word, score in most_similar_good:
    print(f"  {word:15s}  {score:.4f}")

print('\n \n') # In order to clearly seperate results

# Check the mostSimilar words to 'bad'
print("Top similar words to 'bad':") 
# Again the same things
most_similar_bad = model.wv.most_similar(positive=["bad"]) 
for word, score in most_similar_bad:
    print(f"  {word:15s}  {score:.4f}")

print('\n \n') # In order to clearly seperate results

# Check the words similar to 'good' but not similar to 'bad'
print("Words similar to 'good' but not similar to 'bad':")
#Here the input negative is for words not similar to a related words
good_not_bad = model.wv.most_similar(positive=["good"], negative=["bad"])
for word, score in good_not_bad:
    print(f"  {word:15s}  {score:.4f}")
print()

print('\n \n') # In order to clearly seperate results

# Check the words similar to 'bad' but not similar to 'good'
print("Words similar to 'bad' but not similar to 'good':")
bad_not_good = model.wv.most_similar(positive=["bad"], negative=["good"])
for word, score in bad_not_good:
    print(f"  {word:15s}  {score:.4f}")

    

Similarity between "good" and "bad": 0.7342

Top similar words to 'good':
  nice             0.9153
  superb           0.9068
  awesome          0.9057
  excellent        0.9023
  beautiful        0.8899
  okay             0.8854
  gud              0.8829
  perfect          0.8781
  fantastic        0.8688
  decent           0.8682

 

Top similar words to 'bad':
  poor             0.8454
  pathetic         0.7778
  worst            0.7687
  bed              0.7643
  nd               0.7576
  wrost            0.7541
  ........         0.7528
  .......          0.7516
  satisfactory     0.7510
  sad              0.7477

 

Words similar to 'good' but not similar to 'bad':
  wise             0.4420
  look             0.3945
  specification    0.3918
  range            0.3879
  cool             0.3796
  quite            0.3791
  nice             0.3742
  price            0.3708
  premium          0.3641
  feature          0.3595


 

Words similar to 'bad' but not similar to 'good':
  cus

### Q1.B - discussion

**1. Similarity between "good" and "bad"**
The similarity score is high, which is expected in Word2Vec models. Although "good" and "bad" are antonyms, they share the same syntactic context. They appear in identical sentence structures (e.g., "The battery is good" vs. "The battery is bad"), causing their vectors to be close in the embedding space. We can also infer that word2vec word embeddings DOES NOT give information about synonyms and antonyms. It gives information about similarity.

**2. Discussion of Similar Words**
* **Neighbors of "good":** The model successfully clusters positive adjectives like 'nice', 'superb', and 'awesome'. It also captures internet slang such as 'gud' (good), reflecting the informal nature of the dataset. Also it is important that the neighbours 'good' have really high similarity to 'good'. This will be useful later.
* **Neighbors of "bad":** The list includes strong negative adjectives like 'poor' and 'pathetic'. Also compared 'good' and its neighbours, the similarity between the word 'bad' and its neighbours are quite low. This means that the model confidently identify words similar to 'good' instead of 'bad'.

**3. Vector Arithmetic (Isolating Sentiment)**
* **Good - Bad (Pure Positivity):** Subtracting the 'bad' vector highlights words related to positivty rather than just general praise. Words like 'wise', and 'price' appear, suggesting that a key driver for 'good' reviews is value for money. Also we get words like 'nice', 'wise' or 'impressive' which is expected since the they all indicate positive meaning.
* **Bad - Good (Pure Negativity):** Subtracting the 'good' vector isolates unique negative experiences. Words like 'customer', 'care' and 'reply' appear here. This indicates that while 'bad' describes hardware, the uniquely negative experiences are often related to customer service (lack of reply). It is important to note that compare to 'good'-'bad', here we do not have words with negative meaning such as 'poor' or 'pathetic'. I believe this is due to the fact that model is more confident when predicting words similar to 'good' than to the word 'bad'.

**4. Can we identify specific product features?**
Yes, by examining the neighbors of 'bad', we can identify that Customer Care ('customer') and ('care') are the primary pain points for this product. Conversely, the neighbors of 'good' (minus 'bad') suggest that Price/Value ('budget', 'wise') is a specific positive feature.

### Q1.C - representation (15 points)

An important use of word embeddings is representing "documents" (reviews in our case). For this question, before creating the representations, do the following:

- Randomly sample 2 reviews from sentiment label 0, refer to them as sent0_a and sent0_b.
- Randomly sample 2 reviews from sentiment label 1, refer to them as sent1_a and sent1_b.

After the sampling, follow these steps to represent each review:

- Preprocess the review with the given `preprocess_review` function.
- For each token in the review, fetch the vector of that token.
- Take the average of the token vectors in the review to represent that review.

Then, calculate and report the cosine similarity of the two vectors representing:
    - sent0_a and sent0_b
    - sent0_a and sent1_a
    - sent1_a and sent1_b

Does this representation work to capture the labels of the reviews? Do you think there is a better way to represent each review instead of taking the average of the word vectors? Discuss your findings with respect to these questions. Repeating the sampling process several times might give you a better insight.

#### Notes and tips

- You can use `numpy` for your calculations.

In [119]:
# Q1.C - implementation
# you can add cells below if needed

# Get the sentiment 0 reviews
sent0 = train[train["sentiment"] == 0] # This gives both sentiments and reviews
sent0_samples = sent0.sample(2) # Randomly sample 2 reviews

# Get the sentiment 1 reviews
sent1 = train[train["sentiment"] == 1] # This gives both sentiments and reviews
# Here I did not set random_state=0 or 1 since each time I run this I want to get a different randomly selected pair
sent1_samples = sent1.sample(2) # Randomly sample 2 reviews

# Here when I used sent0_samples[0] I got an error. So when I looked it up the trick is to use iloc[0]
sent0_a = sent0_samples.iloc[0]["review"]
sent0_b = sent0_samples.iloc[1]["review"]
sent1_a = sent1_samples.iloc[0]["review"]
sent1_b = sent1_samples.iloc[1]["review"]

print(sent0_a)
print(sent0_b)
print(sent1_a)
print(sent1_b)

print('\n')

# Helper function to get average embedding for a review
def calculate_average_tokens(review_text, model):
    tokens = preprocess_review(review_text)
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    
    if len(vectors) == 0:
        # If no token has a vector (very rare), return a zero vector
        return np.zeros(model.vector_size)
    
    return np.mean(vectors, axis=0)


# Helper function to calculate cosine similarity between two vectors
def cosine_similarity(v1, v2):
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return float(np.dot(v1, v2) / (norm1 * norm2))


# Compute review vectors using the helper method
sent0_a_vec = calculate_average_tokens(sent0_a, model)
sent0_b_vec = calculate_average_tokens(sent0_b, model)
sent1_a_vec = calculate_average_tokens(sent1_a, model)
sent1_b_vec = calculate_average_tokens(sent1_b, model)


# 5) Compute requested cosine similarities
sim_00 = cosine_similarity(sent0_a_vec, sent0_b_vec)  # 0 vs 0
sim_01 = cosine_similarity(sent0_a_vec, sent1_a_vec)  # 0 vs 1
sim_11 = cosine_similarity(sent1_a_vec, sent1_b_vec)  # 1 vs 1

print(f"cosine(sent0_a, sent0_b) = {sim_00:.4f}")
print(f"cosine(sent0_a, sent1_a) = {sim_01:.4f}")
print(f"cosine(sent1_a, sent1_b) = {sim_11:.4f}")


Do not buy this phone, got hanged and heated
Camera is very disappointing and after updating to latest versions front camera loses the quality and music out was very poor as compared to k3note, k4 note and there is an option for slow motion video which is the worst feature in this phone
best for under 12k
Awsm product......


cosine(sent0_a, sent0_b) = 0.8104
cosine(sent0_a, sent1_a) = 0.7138
cosine(sent1_a, sent1_b) = 0.7352


### Q1.C - discussion

**Does this representation work to capture the labels?**
Based on the results, the simple averaging of word vectors is moderately effective but inconsistent.
* **Expectation:** We expect `Sim(Neg, Neg)` and `Sim(Pos, Pos)` to be high, and `Sim(Neg, Pos)` to be low.
* **Reality:** Often, the similarity between *any* two reviews is relatively high (e.g., > 0.60). This is because both positive and negative reviews share the same domain vocabulary (e.g., "phone", "battery", "price", "delivery"). When we average all vectors, these common nouns dominate the vector direction, washing out the sentiment-specific adjectives like "good" or "bad". Also the results are really inconsistent, for each different run, I get a very different result. For most of the results, similarity scores between same sentiment reviews are higher but for some cases the similarity between average vectors from different sentiment reviews are higher. This actually shows that there are better ways for representing documents than Word2vec model. It might be that Word2vec is strong for representing meaning of words but not for representing meanings for documents. 

### Q1.D - training and comparing classifiers (15 points)

For this task, you will use the `user_review_train.csv` and `user_review_test.csv` files to train a binary classification model with Word2Vec representations, and compare its performance with a binary classifier using Bag-of-Words representation.

As the Bag-of-Words classifier, you can either choose the best performing classifier you have implemented in Question 3 of Programming Assignment 1, or you can follow these steps:

- Preprocess the review with the given `preprocess_review` function.
- Order all unique tokens by frequency, take the most frequent 100.
- Use these 100 words as the corpus for Bag-of-Words representation.

For the Word2Vec model, represent the reviews by following these steps:

- Preprocess the review with the given `preprocess_review` function.
- For each token in the review that is also in the most frequent 100 tokens, fetch the vector of that token.
- Take the average of the token vectors selected to represent that review.

After training both classifiers on `user_review_train.csv`, test them with `user_review_test.csv` and report the performance of your models with four metrics: accuracy, precision, recall and F1-score. Compare the performance of both models and discuss in detail.

#### Notes and tips

- You can use `CountVectorizer` from `scikit-learn` or any other library available for Bag-of-Words representation.
- You should select a classification method from the following set of classifiers: `[Naive Bayes, Support Vector Machine, Logistic Regression, Random Forest]`. You can use `scikit-learn`, `nltk`, or any other library for the classifier implementations. 
- You should **not** use the test set `user_reviews_test.csv` during your training process. You should use `user_reviews_train.csv` only.
- You may add a validation step in your training process. To do this, you can further split the `user_reviews_train.csv` data and apply k-fold cross validation.

In [131]:
# Q1.D - implementation
# you can add cells below if needed

# Read the test data
test = pd.read_csv("data/user_review_test.csv")

# First preprocess each review
# Create a tokens column for both train and test data
train['tokens'] = train['review'].apply(preprocess_review)
test['tokens'] = test['review'].apply(preprocess_review)

# Get all words in a single list since we are going to use bag of words
all_train_words = [word for tokens in train['tokens'] for word in tokens]
word_counts = Counter(all_train_words) # this returns word, count pair
top_100_tuples = word_counts.most_common(100) # extract the most common 100 words
top_100_words = [word for word, count in top_100_tuples] # extract only the words from tuples


# Initialize CountVectorizer with only the top 100 vocabulary
vectorizer = CountVectorizer(vocabulary=top_100_words)

# Prepare Features (X) and Sentiments (y)
# Join tokens back to string because CountVectorizer expects strings
train_corpus = [" ".join(tokens) for tokens in train['tokens']]
test_corpus = [" ".join(tokens) for tokens in test['tokens']]

# Transform text into vector of numbers (where each dimension of the vector is the most common tokens)
X_train_bow = vectorizer.transform(train_corpus)
X_test_bow = vectorizer.transform(test_corpus)

y_train = train['sentiment']
y_test = test['sentiment']

# Train Classifier (Logistic Regression)
classifier_bow = LogisticRegression(random_state=42, max_iter=1000)
classifier_bow.fit(X_train_bow, y_train)

# Predict
y_pred_bow = classifier_bow.predict(X_test_bow)

# Metrics
print("Results for Bag-of-Words:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_bow):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_bow):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_bow):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_bow):.4f}")

# Helper function to get w2v vectors that only uses the most common 100 tokens
def get_filtered_w2v_vector(tokens, model, valid_vocab):
    # Keep word only if it is in Top 100 and in Word2Vec model
    valid_tokens = [word for word in tokens if word in valid_vocab and word in model.wv]
    
    if not valid_tokens:
        return np.zeros(model.vector_size)
    
    # Fetch vectors
    vectors = [model.wv[word] for word in valid_tokens]
    
    # Return Average
    return np.mean(vectors, axis=0)

# Generate Features
X_train_w2v = np.array([get_filtered_w2v_vector(t, model, top_100_words) for t in train['tokens']])
X_test_w2v = np.array([get_filtered_w2v_vector(t, model, top_100_words) for t in test['tokens']])

# Train Classifier (Logistic Regression)
classifier_w2v = LogisticRegression(random_state=42, max_iter=1000)
classifier_w2v.fit(X_train_w2v, y_train)

# Predict
y_pred_w2v = clf_w2v.predict(X_test_w2v)

# Metrics
print("Results for Word2Vec:")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_w2v):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_w2v):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_w2v):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_w2v):.4f}")

Results for Bag-of-Words:
Accuracy:  0.7731
Precision: 0.8630
Recall:    0.7196
F1 Score:  0.7848
Results for Word2Vec:
Accuracy:  0.7600
Precision: 0.8555
Recall:    0.7009
F1 Score:  0.7705


### Q1.D - discussion

**Performance Summary**
- Bag-of-Words (BoW): Accuracy 77.3% | F1 Score 0.78
- Word2Vec (W2V): Accuracy 76.0% | F1 Score 0.77 

**Observation:**
The Bag-of-Words model outperformed the Word2Vec model across all reported metrics (Accuracy, Precision, and Recall), though the margin is relatively small.

At first I thought that the advanced method (word2vec) would outperform the simpler one (bag of words), but given the constraints on the task, this behaviour is expected due to the fact that in our Word2Vec approach, we represented a review by taking the average of its word vectors. This causes a loss of information.
- Example: If a review says "good screen, bad battery", the vector for "good" (positive direction) and "bad" (negative direction) might mathematically cancel each other out when averaged, resulting in a neutral vector.
- BoW Advantage: Bag of words keeps these counts separate ('good':1, 'bad':1). The Logistic Regression can then learn that "good" contributes +X to the score and "bad" contributes -Y, preserving both signals.

As I have guessed in part C, While Word2Vec is generally a more powerful representation for capturing semantics, Bag-of-Words is often superior for simple sentiment analysis tasks when using linear classifiers and restricted vocabularies. The explicit presence/absence of specific sentiment keywords (captured by BoW) is often a stronger signal than the average semantic direction (captured by averaged W2V).

## Q2 - Neural Networks for Binary Classification (50 points)

For this task, you will use the `user_review_train.csv` and `user_review_test.csv` files to train two neural network models for the binary classification of user reviews and compare their performances. You are expected to train RNN (part A - 20 points) and TextCNN (part B - 20 points) models, and report the following: 

- Confusion matrix of both models
- Time it took to train both models
- Accuracy, precision, recall, and F1-score of both models
- Other metrics you think are important

Finally (part C), you should discuss the performance of the models according to your reported results. Try to analyse the models in terms of pros and cons of using each one.

#### Notes and tips

- For the embedding layers of the models, you are free to use word embedding methods or leave them randomly initialised. Similarly, you can use word-based or character-based embeddings. However, make sure to explain your decisions.
- You are expected to use `tensorflow` for your implementations, but you can use other libraries if you already have a working setup.

In [25]:
# Preparation step that is required for both Q2.A and Q2.B

# Load the training and test datasets
# The reason I read these inputs again is because I can run directly this cell withohut running previous ones
train = pd.read_csv("data/user_review_train.csv")
test  = pd.read_csv("data/user_review_test.csv")

# Preprocess each review and get the tokens
# Here train_tokens is a list of token lists
train_tokens = [preprocess_review(str(r)) for r in train_df["review"]]
test_tokens  = [preprocess_review(str(r)) for r in test_df["review"]]

counter = Counter()
for t in train_tokens:
    counter.update(t)

max_vocab = 10000    # Get the most common 10000 words in order to get rid of very uncommon words
most_common = counter.most_common(max_vocab)

# Here I am assigning each word an index so that I can directly get the embedding vector of that vector
word_index = {w: i+1 for i, (w, c) in enumerate(most_common)}
vocab_size = len(word_index) + 1 # Since indexing start from 0

# Helper function to convert a list of words (tokens) into a list of integers (word indexes).
def to_sequence(tokens):
    return [word_index[w] for w in tokens if w in word_index]

# Use this to store the reviews as a list of word indexes
review_train_seq = [to_sequence(t) for t in train_tokens]
review_test_seq  = [to_sequence(t) for t in test_tokens]

maxlen = 100  # fixed-length sequence
# For LSTM, the input size must be same for each review, so for shorter review, we do padding and for long reviews we truncate so that 
# we get a fixed size input
review_train_pad = pad_sequences(review_train_seq, maxlen=maxlen, padding="post", truncating="post")
review_test_pad  = pad_sequences(review_test_seq,  maxlen=maxlen, padding="post", truncating="post")

print("Train padded shape:", review_train_pad.shape)
print("Test padded shape :", review_test_pad.shape)
print("Vocabulary size   :", vocab_size)

Train padded shape: (11751, 100)
Test padded shape : (1675, 100)
Vocabulary size   : 10001


In [42]:
# Q2.A - implementation of RNN
# you can add cells below if needed

# The number of features in each word vector
embedding_dim = 100 

rnn_model = Sequential([ # # Sequential means each layer feeds directly into the next
    Embedding(vocab_size, embedding_dim), # Converts integer-encoded words into dense embedding vectors
    # Bidirectional LSTM to capture context from both left and right.
    Bidirectional(LSTM(64)), # Use LSTM to keep the meaning of the previous words in the memory and 
    Dense(64, activation='relu'), # A small dense layer to learn non-linear combinations of LSTM features
    Dense(1, activation="sigmoid") # Produces a probability between 0 and 1 indicating sentiment
])

# metrics = accuracy -> track accuracy during training
# user binary cross entropy for calculating the loss
rnn_model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=1e-3), metrics=["accuracy"])

# Start the timer to calculate the training time of RNN
start = time.time()

# Train the model with padded input and sentiments
# Here I used padded input for RNN aswell because the training done in batches
rnn_model.fit(review_train_pad, sentiment_train, epochs=5, batch_size=64, validation_split=0.1, verbose=1)

# Calculate the total training time
rnn_time = time.time() - start

# Make the model predict sentiments
rnn_pred = (rnn_model.predict(review_test_pad) > 0.5)

print("RNN Training Time:", rnn_time)
print("Accuracy:",  accuracy_score(sentiment_test, rnn_pred))
print("Precision:", precision_score(sentiment_test, rnn_pred))
print("Recall:",    recall_score(sentiment_test, rnn_pred))
print("F1:",        f1_score(sentiment_test, rnn_pred))
print("Confusion Matrix:\n", confusion_matrix(sentiment_test, rnn_pred))

Epoch 1/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.8048 - loss: 0.4241 - val_accuracy: 0.7509 - val_loss: 0.5223
Epoch 2/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 13s 78ms/step - accuracy: 0.8911 - loss: 0.2793 - val_accuracy: 0.7874 - val_loss: 0.4641
Epoch 3/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 13s 77ms/step - accuracy: 0.9087 - loss: 0.2381 - val_accuracy: 0.7517 - val_loss: 0.5719
Epoch 4/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 13s 78ms/step - accuracy: 0.9211 - loss: 0.2108 - val_accuracy: 0.7781 - val_loss: 0.5645
Epoch 5/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 13s 77ms/step - accuracy: 0.9349 - loss: 0.1800 - val_accuracy: 0.7636 - val_loss: 0.6645
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
RNN Training Time: 62.126697063446045
Accuracy: 0.8370149253731344
Precision: 0.8920454545454546
Recall: 0.8151609553478713
F1: 0.8518719479110146
Confusion Matrix:
 [[617  95]
 [178 785]]


In [41]:
# Q2.B - TextCNN model for sentiment classification

# Build a simple TextCNN model using Sequential, where each layer feeds directly into the next
cnn_model = Sequential([
    Embedding(vocab_size, embedding_dim), # Converts integer-encoded words into dense embedding vectors
    # Applies 128 filters that slide over the sequence of embeddings
    # kernel_size=5 means each filter looks at a window of 5 consecutive words (like detecting a phrase)
    Conv1D(filters=128, kernel_size=5, activation="relu"),
    # Takes the maximum value of each filter over the entire sequence
    # Reduces the sequence to a fixed-size vector regardless of sentence length
    GlobalMaxPooling1D(),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])

# Same with RNN example
cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Start timer for measuring CNN training time
start = time.time()

# Train the TextCNN on padded sequences
# the batches still require fixed-size tensors, so padding is still necessary
cnn_model.fit(review_train_pad, sentiment_train, epochs=5, batch_size=64, validation_split=0.1)

# Measure total training time for CNN
cnn_time = time.time() - start

# Make predictions on the test set
# The model outputs probabilities, so we threshold > 0.5 to get binary predictions
cnn_pred = (cnn_model.predict(review_test_pad) > 0.5)

print("CNN Training Time:", cnn_time)
print("Accuracy:",  accuracy_score(sentiment_test, cnn_pred))
print("Precision:", precision_score(sentiment_test, cnn_pred))
print("Recall:",    recall_score(sentiment_test, cnn_pred))
print("F1:",        f1_score(sentiment_test, cnn_pred))
print("Confusion Matrix:\n", confusion_matrix(sentiment_test, cnn_pred))

Epoch 1/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8000 - loss: 0.4289 - val_accuracy: 0.7491 - val_loss: 0.5721
Epoch 2/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8944 - loss: 0.2701 - val_accuracy: 0.8010 - val_loss: 0.4798
Epoch 3/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9262 - loss: 0.1966 - val_accuracy: 0.7696 - val_loss: 0.6458
Epoch 4/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9545 - loss: 0.1354 - val_accuracy: 0.7747 - val_loss: 0.7046
Epoch 5/5
166/166 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9666 - loss: 0.0992 - val_accuracy: 0.7959 - val_loss: 0.6913
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
CNN Training Time: 11.416546821594238
Accuracy: 0.8226865671641791
Precision: 0.8588362068965517
Recall: 0.8276220145379024
F1: 0.8429402432575357
Confusion Matrix:
 [[581 131]
 [166 797]]


### Q2.C - discussion

**Performance Summary**


| Metric            | RNN (BiLSTM) | TextCNN |
|------------------|--------------|---------|
| **Training Time** | 62.13 s      | 11.42 s |
| **Accuracy**      | 0.8370       | 0.8227  |
| **Precision**     | 0.8920       | 0.8588  |
| **Recall**        | 0.8152       | 0.8276  |
| **F1-score**      | 0.8519       | 0.8429  |

It seems that RNN model is slightly accurate compared to TextCNN model due to his high presicion. However, its training time is way longer and has a slightly lower recall value.

**Interpretations of the Results**

- The Bidirectional LSTM captures sequential information from both the left and right directions. This helps the model understand word ordering, resulting in a higher accuracy.
    
- From the confusion matrix and the metric table, we can infer that even though RNN is successful at predicting positive sentiments, it can miss some positive sentiments (since it has high precision but slightly lower recall).

- However, RNN model's training time is significantly higher than the CNN model. This is due to the fact that standard RNN processing is sequential, where the hidden state $h_t$ depends on $h_{t-1}$. This sequential dependency makes parallelization difficult, resulting in the longer training time.

- Whereas the Convolutional layer computes features for all windows across the sentence simultaneously using tensor operations which makes the training highly parallelizable.

- CNN model achieved higher recall than the RNN, meaning it detected more true positives, but at the cost of more false positives.

**Impact of Design Choices**

- During training, I tried to keep the conditions for both model as close as possible since we are not interested in model parameters but the actual model choice.

- Randomly initialized Embedding Layer:
    - Both models learn task-specific word embeddings during training.
    - This is suitable given the dataset size.

- Sequence Padding (maxlen = 100):
    - Required because batching in TensorFlow needs fixed-length inputs.
    - 100 tokens reasonably covers all words.
    - Longer reviews are truncated; shorter reviews padded. For some reviews, this resulted in loss of meaning (due to truncation) and meaningless sentences (due to extreme padding).

- Bidirectional LSTM:
    - Learns context in both directions -> higher accuracy and precision.
    - Doubling the computation -> much slower training.

- CNN with Conv1D + GlobalMaxPooling:
    - Learns n-gram patterns -> good recall.
    - Loses ordering information after pooling -> slightly worse precision.
    - Extremely fast training due to parallelism.

**Final Comparison of Models**

                     RNN (BiLSTM)             TextCNN
    ---------------------------------------------------------------
    Captures word order?     Yes                    No
    Captures long-range?     Yes                    No
    Detects local phrases?   Moderate               Excellent
    Training speed           Slow (62s)             Fast (11s)
    Precision                Higher                 Lower
    Recall                   Slightly lower         Higher
    Best for                 Complex text           Short/simple reviews

**Conclusion**

The RNN achieved the best overall performance (highest accuracy, precision, and F1-score) because it captures detailed context and long-distance dependencies in reviews.

The TextCNN, while slightly less accurate, trained nearly six times faster and still produced strong results. Its high recall indicates it effectively identifies positive sentiment when clear cues exist.


• Choose RNN when accuracy and interpretability matter.
• Choose CNN when speed and scalability are more important.

Both models behaved as expected based on their architecture, and both represent 
valid approaches for sentiment classification on this dataset.
"""